In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import io
import requests
from datetime import datetime

In [3]:
# Fetch data from URL
url = "https://hebbkx1anhila5yf.public.blob.vercel-storage.com/power_consumption_data-8TGq532mDXrPjPnTX0V7a3cgGOlxXT.csv"
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text))

In [14]:

# Convert datetime to proper datetime format
#data['datetime'] = pd.to_datetime(data['datetime'])
#data.set_index('datetime', inplace=True)

# Convert numeric columns to float
numeric_cols = ['Global_active_power', 'Global_reactive_power', 'Voltage', 
                'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 
                'Sub_metering_3', 'temp', 'humidity']
for col in numeric_cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Resample to daily data
daily_data = data.copy()

# Fill missing values
daily_data = daily_data.fillna(method='ffill')

# By day of week
daily_avg = daily_data['Global_active_power'].groupby(daily_data.index.dayofweek).mean()
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig = px.bar(x=days, y=daily_avg.values, 
             labels={'x': 'Day of Week', 'y': 'Average Global Active Power'},
             title='Average Power Consumption by Day of Week')
fig.update_layout(template='plotly_white')
fig.show()

# By month
monthly_avg = daily_data['Global_active_power'].groupby(daily_data.index.month).mean()
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig = px.line(x=months, y=monthly_avg.values, 
              labels={'x': 'Month', 'y': 'Average Global Active Power'},
              title='Average Power Consumption by Month')
fig.update_layout(template='plotly_white')
fig.show()

# Analyze relationship between power consumption and environmental factors
# Temperature vs Power Consumption
fig = px.scatter(daily_data, x='temp', y='Global_active_power', 
                 trendline='ols',
                 labels={'temp': 'Temperature', 'Global_active_power': 'Global Active Power'},
                 title='Temperature vs Power Consumption')
fig.update_layout(template='plotly_white')
fig.show()

# Humidity vs Power Consumption
fig = px.scatter(daily_data, x='humidity', y='Global_active_power', 
                 trendline='ols',
                 labels={'humidity': 'Humidity', 'Global_active_power': 'Global Active Power'},
                 title='Humidity vs Power Consumption')
fig.update_layout(template='plotly_white')
fig.show()

# Analyze sub-metering data
sub_metering_data = daily_data[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
sub_metering_data.columns = ['Kitchen', 'Laundry Room', 'Water Heater & AC']

fig = go.Figure()
for column in sub_metering_data.columns:
    fig.add_trace(go.Scatter(x=sub_metering_data.index, y=sub_metering_data[column],
                             mode='lines', name=column))

fig.update_layout(title='Sub-metering Power Consumption Over Time',
                  xaxis_title='Date',
                  yaxis_title='Power (watt-hour)',
                  template='plotly_white',
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

# Calculate percentage of total consumption by sub-meter
total_sub_metering = sub_metering_data.sum(axis=1)
sub_metering_pct = sub_metering_data.sum() / total_sub_metering.sum() * 100

fig = px.pie(values=sub_metering_pct, names=sub_metering_pct.index,
             title='Percentage of Power Consumption by Area')
fig.update_layout(template='plotly_white')
fig.show()

# Anomaly detection using IQR method
def detect_anomalies(data, column, threshold=1.5):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    
    anomalies = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return anomalies, lower_bound, upper_bound

anomalies, lower_bound, upper_bound = detect_anomalies(daily_data, 'Global_active_power')

fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_data.index, y=daily_data['Global_active_power'],
                         mode='lines', name='Power Consumption'))
fig.add_trace(go.Scatter(x=anomalies.index, y=anomalies['Global_active_power'],
                         mode='markers', name='Anomalies', marker=dict(color='red', size=8)))

fig.add_shape(type="line", x0=daily_data.index[0], y0=upper_bound, x1=daily_data.index[-1], y1=upper_bound,
              line=dict(color="red", width=2, dash="dash"))
fig.add_shape(type="line", x0=daily_data.index[0], y0=lower_bound, x1=daily_data.index[-1], y1=lower_bound,
              line=dict(color="red", width=2, dash="dash"))

fig.update_layout(title='Anomaly Detection in Power Consumption',
                  xaxis_title='Date',
                  yaxis_title='Global Active Power',
                  template='plotly_white')
fig.show()

print(f"Number of anomalies detected: {len(anomalies)}")
print(f"Percentage of anomalies: {len(anomalies) / len(daily_data) * 100:.2f}%")

# Feature importance analysis for power consumption
from sklearn.ensemble import RandomForestRegressor

# Prepare features and target
features = daily_data[['Global_reactive_power', 'Voltage', 'temp', 'humidity']]
target = daily_data['Global_active_power']

# Add time-based features
features['month'] = daily_data.index.month
features['day_of_week'] = daily_data.index.dayofweek
features['is_weekend'] = (features['day_of_week'] >= 5).astype(int)

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(features, target)

# Get feature importances
importances = pd.DataFrame({
    'Feature': features.columns,
    'Importance': rf_model.feature_importances_
})
importances = importances.sort_values('Importance', ascending=False)

fig = px.bar(importances, x='Feature', y='Importance',
             title='Feature Importance for Power Consumption')
fig.update_layout(template='plotly_white')
fig.show()

print("Feature Importance Analysis:")
print(importances)

# Forecast future power consumption (using the best model from previous analysis)
# For demonstration, let's use Prophet as it's good for long-term forecasting

from prophet import Prophet
from datetime import timedelta

# Prepare data for Prophet
prophet_data = pd.DataFrame({'ds': daily_data.index, 'y': daily_data['Global_active_power'].values})
prophet_data['temp'] = daily_data['temp'].values
prophet_data['humidity'] = daily_data['humidity'].values

# Train Prophet model
model = Prophet(
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10,
    holidays_prior_scale=10,
    seasonality_mode='multiplicative',
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True
)
model.add_regressor('temp')
model.add_regressor('humidity')
model.fit(prophet_data)

# Create future dataframe for forecasting (next 30 days)
future = model.make_future_dataframe(periods=30)

# Create 'temp' and 'humidity' columns initialized to NaN
future['temp'] = np.nan
future['humidity'] = np.nan

# Compute day-of-year averages across all years
day_of_year = daily_data.index.strftime('%m-%d')
temp_avg_by_day = daily_data.groupby(day_of_year)['temp'].mean().to_dict()
humidity_avg_by_day = daily_data.groupby(day_of_year)['humidity'].mean().to_dict()

# Fallback values
fallback_temp = daily_data['temp'].mean()
fallback_humidity = daily_data['humidity'].mean()

# Fill only last 30 rows (future dates)
future_dates = future['ds'].iloc[-30:]
future_keys = future_dates.dt.strftime('%m-%d')

future.loc[future.index[-30:], 'temp'] = [temp_avg_by_day.get(k, fallback_temp) for k in future_keys]
future.loc[future.index[-30:], 'humidity'] = [humidity_avg_by_day.get(k, fallback_humidity) for k in future_keys]

# ✅ Final check: fill any remaining NaNs (just in case)
future['temp'].fillna(fallback_temp, inplace=True)
future['humidity'].fillna(fallback_humidity, inplace=True)


# Make forecast
forecast = model.predict(future)


from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(model, initial='365 days', period='180 days', horizon='30 days')
df_p = performance_metrics(df_cv)
print(df_p[['horizon', 'mape', 'rmse']])


# Plot forecast
fig = go.Figure()
fig.add_trace(go.Scatter(x=prophet_data['ds'], y=prophet_data['y'],
                         mode='lines', name='Historical', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=forecast['ds'][-30:], y=forecast['yhat'][-30:],
                         mode='lines', name='Forecast', line=dict(color='red')))
fig.add_trace(go.Scatter(x=forecast['ds'][-30:], y=forecast['yhat_lower'][-30:],
                         fill=None, mode='lines', line=dict(color='rgba(255,0,0,0.2)'), showlegend=False))
fig.add_trace(go.Scatter(x=forecast['ds'][-30:], y=forecast['yhat_upper'][-30:],
                         fill='tonexty', mode='lines', line=dict(color='rgba(255,0,0,0.2)'),
                         name='Prediction Interval'))

fig.update_layout(title='30-Day Power Consumption Forecast',
                  xaxis_title='Date',
                  yaxis_title='Global Active Power',
                  template='plotly_white',
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

print("\nAnalysis and recommendations completed.")

C:\Users\bhusa\AppData\Local\Temp\ipykernel_30496\2260024687.py:16: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Number of anomalies detected: 104
Percentage of anomalies: 7.22%


C:\Users\bhusa\AppData\Local\Temp\ipykernel_30496\2260024687.py:121: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bhusa\AppData\Local\Temp\ipykernel_30496\2260024687.py:122: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bhusa\AppData\Local\Temp\ipykernel_30496\2260024687.py:123: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

Feature Importance Analysis:
                 Feature  Importance
1                Voltage    0.308350
2                   temp    0.222868
0  Global_reactive_power    0.138539
3               humidity    0.138140
5            day_of_week    0.090592
4                  month    0.083129
6             is_weekend    0.018382


10:27:04 - cmdstanpy - INFO - Chain [1] start processing
10:27:04 - cmdstanpy - INFO - Chain [1] done processing
Seasonality has period of 365.25 days which is larger than initial window. Consider increasing initial.


  0%|          | 0/6 [00:00<?, ?it/s]

10:27:04 - cmdstanpy - INFO - Chain [1] start processing
10:27:05 - cmdstanpy - INFO - Chain [1] done processing
10:27:05 - cmdstanpy - INFO - Chain [1] start processing
10:27:05 - cmdstanpy - INFO - Chain [1] done processing
10:27:05 - cmdstanpy - INFO - Chain [1] start processing
10:27:05 - cmdstanpy - INFO - Chain [1] done processing
10:27:06 - cmdstanpy - INFO - Chain [1] start processing
10:27:06 - cmdstanpy - INFO - Chain [1] done processing
10:27:06 - cmdstanpy - INFO - Chain [1] start processing
10:27:06 - cmdstanpy - INFO - Chain [1] done processing
10:27:07 - cmdstanpy - INFO - Chain [1] start processing
10:27:07 - cmdstanpy - INFO - Chain [1] done processing


   horizon      mape      rmse
0   3 days  0.595634  0.514961
1   4 days  0.615727  0.482452
2   5 days  0.587067  0.511245
3   6 days  0.629991  0.668350
4   7 days  0.796731  0.651281
5   8 days  0.683450  0.623205
6   9 days  0.722815  0.398271
7  10 days  0.646366  0.794510
8  11 days  0.826057  0.819130
9  12 days  0.855561  0.834735
10 13 days  0.793143  0.430927
11 14 days  0.594781  0.363337
12 15 days  0.511282  0.321774
13 16 days  0.523953  0.413891
14 17 days  0.516220  0.509523
15 18 days  0.504827  0.779828
16 19 days  0.510216  0.788366
17 20 days  0.572755  0.699892
18 21 days  0.589073  0.525407
19 22 days  0.570116  0.526139
20 23 days  0.607254  0.531149
21 24 days  0.597932  0.628546
22 25 days  0.566159  0.619803
23 26 days  0.517386  0.664938
24 27 days  0.542932  0.431484
25 28 days  0.540140  0.396885
26 29 days  0.610575  0.416293
27 30 days  0.560222  0.471403



Analysis and recommendations completed.
